<a href="https://colab.research.google.com/github/IshaSarangi/LangChain_Chatbot/blob/main/LangChain_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Building A Chatbot

We will go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

*   Conversational RAG: Enable a chatbot experience over an external source of data
*   Agents: Build a chatbot that can take actions



In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()   #Loading all the environment variable
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq
# model = ChatGroq(model='Gemma2-9b-It', groq_api_key=groq_api_key)
model = ChatGroq(model='llama-3.1-8b-instant', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7de0d02e6270>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7de0acfaef60>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Isha and I am an Gen AI enthusiast. I live in Cuttack, Odisha")])

AIMessage(content="Nice to meet you, Isha. It's great to hear that you're a Gen AI enthusiast. Odisha is a beautiful state with rich cultural heritage, and Cuttack is a historic city with a lot of interesting stories to tell.\n\nWhat aspects of Gen AI do you find most fascinating? Are you interested in its potential applications, its technological advancements, or perhaps its impact on society? And do you have any favorite AI-related projects or developments that you've come across recently?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 59, 'total_tokens': 157, 'completion_time': 0.183608952, 'prompt_time': 0.00391552, 'queue_time': 0.215016658, 'total_time': 0.187524472}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--cab058d8-55de-4a2c-b340-585369dfc331-0', usage_metadata={'input_tokens': 59, 'output_tokens': 98, 'total

In [ ]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Isha and I am an Gen AI enthusiast. I live in Cuttack, Odisha"),
        AIMessage(content="Namaste Isha! It's great to meet you. I'm happy to learn that you're an enthusiast of Gen AI. Cuttack, the silver city of Odisha, is a beautiful place to live. What sparked your interest in General Artificial Intelligence (Gen AI), and what do you hope to achieve or learn about it?"),
        #HumanMessage(content="Hey What's my name and what do I do?")
        HumanMessage(content="Which city is known as the silver city of Odisha?")
    ]
)

AIMessage(content='Cuttack is actually known as "The Millennium City" or "The Silver City of Odisha". The city\'s name is often associated with its beautiful silver filigree work, which is a famous tradition in Odisha. The city has a rich history and is a significant cultural hub in the state.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 149, 'total_tokens': 212, 'completion_time': 0.117899316, 'prompt_time': 0.008420354, 'queue_time': 0.2228159, 'total_time': 0.12631967}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2de56fa1-9b91-48a1-8f79-2605e75d20bd-0', usage_metadata={'input_tokens': 149, 'output_tokens': 63, 'total_tokens': 212})

###Message History

We can use a message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {'configurable': {'session_id': 'chat1'}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Isha and I am an Gen AI enthusiast. I live in Cuttack, Odisha")],
    config=config
)

In [ ]:
response.content

"Nice to meet you, Isha. It's great to hear that you're a Gen AI enthusiast. Odisha is a beautiful state with a rich cultural heritage. Cuttack, being the former capital of Odisha, has a lot of historical significance. \n\nHow did you get interested in Gen AI, and what aspects of it fascinate you the most?"

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='Your name is Isha.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 147, 'total_tokens': 154, 'completion_time': 0.011141849, 'prompt_time': 0.010978181, 'queue_time': 0.356085964, 'total_time': 0.02212003}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--eac3a63c-a5d6-44b4-aa47-ca58c9c25149-0', usage_metadata={'input_tokens': 147, 'output_tokens': 7, 'total_tokens': 154})

In [ ]:
#change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name. This is the beginning of our conversation, and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to know it."

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Isha")],
    config=config1
)
response.content

'Nice to meet you, Isha. How are you doing today?'

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Isha. I remember from earlier in our conversation.'

###Prompt Templates

Prompt Templates help to run raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we will add in more input besides just the messages.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful assisstant. Answer all questions to the best of your ability.'),
        MessagesPlaceholder(variable_name='messages')
    ]
)

chain = prompt|model

In [ ]:
chain.invoke({'messages':[HumanMessage(content='Hi. My name is Isha.')]})

AIMessage(content="Nice to meet you, Isha. I'm happy to assist you with any questions or topics you'd like to discuss. Is there something specific on your mind, or would you like to start with a conversation?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 61, 'total_tokens': 105, 'completion_time': 0.072413509, 'prompt_time': 0.004426732, 'queue_time': 0.293362253, 'total_time': 0.076840241}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b14609a0-6741-49bf-8baf-f86d1964c642-0', usage_metadata={'input_tokens': 61, 'output_tokens': 44, 'total_tokens': 105})

In [ ]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {'configurable': {'session_id': 'chat3'}}
response = with_message_history.invoke(
    [HumanMessage(content='Hi. My name is Isha.')],
    config=config
)

response

AIMessage(content="Hello Isha, it's nice to meet you. I'm here to assist you with any questions or topics you'd like to discuss. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 61, 'total_tokens': 97, 'completion_time': 0.041938639, 'prompt_time': 0.003746845, 'queue_time': 0.215219354, 'total_time': 0.045685484}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fcaf9947-3243-4cfd-821e-8a6064df33ed-0', usage_metadata={'input_tokens': 61, 'output_tokens': 36, 'total_tokens': 97})

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Isha.'

In [ ]:
#Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            'You are a helpful assisstant. Answer all questions to the best of your ability. '
        ),
        MessagesPlaceholder(variable_name='messages'),
    ]
)

chain = prompt|model

In [ ]:
response = chain.invoke({'messages': [HumanMessage(content='Hi. My name is Isha.')]})
response.content

"Hello Isha, it's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. How's your day going so far?"

Let's now wrap this more complicated chain in a Message History Class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = 'messages'
)

In [ ]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Isha")],"language":"Hindi"},
    config=config
)
repsonse.content

'Nice to meet you, Isha. How can I assist you today?'

In [ ]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="What is my name?")], "language": "Hindi"},
    config=config,
)
response.content

'Your name is Isha.'

###Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in. "trim_messages" helps to reduce how many messages we are sending to the model. The trimmer allows us to specify how many tokens we want to kepp, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=500,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Isha"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm Isha", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do I like?")],
    "language":"English"
    }
)
response.content

'You like vanilla ice cream!'

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked the math problem "what\'s 2 + 2".'

In [ ]:
#Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="Whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Isha!'

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What math problem did I ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked the math problem: 2 + 2.'